# **DataFrame imdb**

## **Import des bibliothèques nécessaires**

In [1]:
import pandas as pd

## **Chargement des datasets IMDb**

In [2]:
# Nous importons les données directement depuis les fichiers IMDb compressés au format `.tsv.gz`.

#* Datasets principaux
df_title_basics = pd.read_csv('https://datasets.imdbws.com/title.basics.tsv.gz', sep='\t', compression='gzip', na_values='\\N')
df_title_ratings = pd.read_csv('https://datasets.imdbws.com/title.ratings.tsv.gz', sep='\t', compression='gzip', na_values='\\N')

#* Datasets complémentaires
df_title_akas = pd.read_csv('https://datasets.imdbws.com/title.akas.tsv.gz', sep='\t', compression='gzip', na_values='\\N')
df_title_crew = pd.read_csv('https://datasets.imdbws.com/title.crew.tsv.gz', sep='\t', compression='gzip', na_values='\\N')
df_name_basics = pd.read_csv('https://datasets.imdbws.com/name.basics.tsv.gz', sep='\t', compression='gzip', na_values='\\N')
df_title_principals = pd.read_csv('https://datasets.imdbws.com/title.principals.tsv.gz', sep='\t', compression='gzip', na_values='\\N')

C:\Users\koke7\AppData\Local\Temp\ipykernel_29804\3535879025.py:4: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df_title_basics = pd.read_csv('https://datasets.imdbws.com/title.basics.tsv.gz', sep='\t', compression='gzip', na_values='\\N')
C:\Users\koke7\AppData\Local\Temp\ipykernel_29804\3535879025.py:8: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df_title_akas = pd.read_csv('https://datasets.imdbws.com/title.akas.tsv.gz', sep='\t', compression='gzip', na_values='\\N')


## **Nettoyage des datasets IMDb**

In [3]:

#! === SETUP ===
#? title_basics

# 1. Garder uniquement les films (`titleType = 'movie'`).
# 2. Supprimer les lignes avec des valeurs manquantes essentielles.
# 3. Filtrer les années de production pour garder uniquement celles entre 1970 et 2025.
# 4. Réduire le dataset aux colonnes essentielles.

df_title_basics_clean = (
    df_title_basics[df_title_basics['titleType'] == 'movie']
    .dropna(subset=['startYear', 'genres'])
    .assign(startYear=lambda x: x['startYear'].astype(int))
    .query('1970 <= startYear <= 2025')
    [['tconst', 'primaryTitle', 'startYear']]
)


#! === SETUP ===
#? title_rating

# 1. Garder uniquement les films ayant plus de 1000 votes.

df_title_ratings_clean = df_title_ratings[df_title_ratings['numVotes'] > 1000]


#! === SETUP ===
#? title_akas

# 1. Garder uniquement les titres traduits en français (`region = 'FR'`).
# 2. Réduire aux colonnes essentielles et renommer pour correspondre au format attendu.

df_title_akas_clean = (
    df_title_akas[df_title_akas['region'] == 'FR']
    [['titleId', 'title']]
    .rename(columns={'titleId': 'tconst', 'title': 'Titre Français'})
)


#! === SETUP ===
#? title_crew

# 1. Remplir les valeurs manquantes dans `directors` avec "Unknown".
# 2. Transformer `directors` en une liste.
# 3. Exploser les réalisateurs en lignes individuelles pour fusionner avec `name.basics`.
# 4. Regrouper les réalisateurs par film sous forme de liste.

df_title_crew_clean = (
    df_title_crew.fillna({'directors': 'Unknown'})
    .assign(directors=lambda x: x['directors'].str.split(','))
    .explode('directors')
    .merge(df_name_basics[['nconst', 'primaryName']], left_on='directors', right_on='nconst', how='left')
    .groupby('tconst')['primaryName'].apply(list).reset_index()
)


#! === SETUP ===
#? title_principals

# 1. Filtrer les acteurs uniquement.
# 2. Fusionner avec `name.basics` pour récupérer les noms des acteurs.
# 3. Regrouper les acteurs par film sous forme de liste.

df_title_principals_clean = (
    df_title_principals.dropna(subset=['category'])
    .query("category == 'actor' or category == 'actress'")
    .merge(df_name_basics[['nconst', 'primaryName']], on='nconst', how='left')
    .groupby('tconst')['primaryName'].apply(list).reset_index()
)

## **Fusion des datasets IMDb**

In [4]:

#! === MERGEUP ===
#* Nous fusionnons les datasets nettoyés pour créer un seul dataframe regroupant toutes les informations utiles.

#* Fusion `title.basics` et `title.ratings`
df_merged_v1 = df_title_basics_clean.merge(df_title_ratings_clean, on='tconst')

#* Fusion avec `title.akas`
df_merged_v2 = df_merged_v1.merge(df_title_akas_clean, on='tconst', how='left')

#* Fusion avec `title.crew`
df_merged_v3 = df_merged_v2.merge(df_title_crew_clean, on='tconst', how='left')

#* Ajout des acteurs depuis `title.principals`
df_merged_v3['Acteurs'] = df_merged_v3['tconst'].map(dict(zip(df_title_principals_clean['tconst'], df_title_principals_clean['primaryName'])))

#* Export du dataframe IMDb final
df_merged_v3.to_csv("../data/raw/df_movie.csv", index=False)

# **Initialisation de TMDB**

## **Chargement des données**

In [9]:
import pandas as pd
import requests

# Clé API TMDb
api_key = 'f26ef44bcadc5d6ffa22263ea37741ce'

# === IMPORT ===
df_movie = pd.read_csv('../data/raw/df_movie.csv')
df_tmdb = pd.read_csv('../data/raw/tmdb_full.csv')

## **Initialisation des datasets**

In [2]:

#* Nous conservons uniquement les colonnes pertinentes pour le projet.
df_tmdb = df_tmdb[
    ['id', 'backdrop_path', 'budget', 'genres', 'imdb_id', 'original_language', 'overview', 'popularity', 
     'poster_path', 'production_countries', 'release_date', 'revenue', 'runtime', 
     'spoken_languages', 'vote_average', 'vote_count', 'production_companies']
]

#* Renommage des colonnes pour le dataset IMDb (df_movie)
df_movie.rename(columns={
    'tconst': 'ID imdb',
    'primaryTitle': 'Titre Original',
    'averageRating': 'Note imdb',
    'numVotes': 'Votes imdb',
    'title': 'Titre Français',
    'actors': 'Acteurs',
    'primaryName': 'Réalisateur(s)'
}, inplace=True)

#* Mise en forme des colonnes "Réalisateur(s)" et "Acteurs" pour convertir les listes en chaînes lisibles.
df_movie['Réalisateur(s)'] = df_movie['Réalisateur(s)'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)
df_movie['Réalisateur(s)'] = df_movie['Réalisateur(s)'].str.strip("[]").str.replace("'", "")

df_movie['Acteurs'] = df_movie['Acteurs'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)
df_movie['Acteurs'] = df_movie['Acteurs'].str.strip("[]").str.replace("'", "")

#* Renommage des colonnes pour le dataset TMDB (df_tmdb)
df_tmdb.rename(columns={
    'id': 'ID tmdb',
    'backdrop_path': 'Image de Fond',
    'budget': 'Budget',
    'genres': 'Genres',
    'overview': 'Synopsis',
    'original_language': 'Langue Originale',
    'popularity': 'Popularité',
    'poster_path': 'Affiche',
    'production_countries': 'Pays de Production',
    'release_date': 'Date de Sortie',
    'revenue': 'Box Office',
    'runtime': 'Durée',
    'spoken_languages': 'Langues Parlées',
    'vote_average': 'Note tmdb',
    'vote_count': 'Votes tmdb',
    'production_companies': 'Compagnies de Production'
}, inplace=True)

#* Mise en forme des colonnes avec des listes pour les rendre lisibles.
df_tmdb['Pays de Production'] = df_tmdb['Pays de Production'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)
df_tmdb['Pays de Production'] = df_tmdb['Pays de Production'].str.strip("[]").str.replace("'", "")

df_tmdb['Compagnies de Production'] = df_tmdb['Compagnies de Production'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)
df_tmdb['Compagnies de Production'] = df_tmdb['Compagnies de Production'].str.strip("[]").str.replace("'", "")

df_tmdb['Langues Parlées'] = df_tmdb['Langues Parlées'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)
df_tmdb['Langues Parlées'] = df_tmdb['Langues Parlées'].str.strip("[]").str.replace("'", "")

df_tmdb['Genres'] = df_tmdb['Genres'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)
df_tmdb['Genres'] = df_tmdb['Genres'].str.strip("[]").str.replace("'", "")

df_movie['Acteurs'] = df_movie['Acteurs'].fillna('').str.split(',').apply(set)
df_movie['Acteurs'] = df_movie['Acteurs'].apply(lambda x: ','.join(set(x)))

df_movie["Titre Français"] = df_movie["Titre Français"].fillna(df_movie["Titre Original"])

## **Fusion de Imdb et Tmdb**

In [3]:

#! === MERGEUP ===

df_final = pd.merge(df_movie, df_tmdb, left_on='ID imdb', right_on='imdb_id')

## **Ajout de données supplémentaires (temps de traitement : long)**

In [11]:

def get_movie_details_from_tmdb(movie_id, language='fr'):
    """
    Récupère les détails du film depuis TMDb dans la langue spécifiée
    """
    url = f"https://api.themoviedb.org/3/movie/{movie_id}?api_key={api_key}&language={language}"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Erreur lors de la récupération des détails du film : {response.status_code}")
        return None


def remplacer_synopsis_par_overview_fr(df):
    """
    Remplace la colonne 'Synopsis' par le synopsis en français (overview_fr) en utilisant l'API TMDb
    """
    # Vérifier les colonnes nécessaires
    required_columns = ['ID tmdb', 'Synopsis']
    for col in required_columns:
        if col not in df.columns:
            print(f"La colonne '{col}' n'existe pas dans le DataFrame.")
            return df

    # Récupérer le synopsis en français depuis TMDb et remplacer la colonne 'Synopsis'
    for index, row in df.iterrows():
        movie_id = row['ID tmdb']  # Assurez-vous que votre DataFrame contient l'ID TMDb
        movie_details = get_movie_details_from_tmdb(movie_id, language='fr')
        if movie_details:
            df.at[index, 'Synopsis'] = movie_details.get('overview', '')

    return df

def get_cast_with_roles(tmdb_id):
    url = f"https://api.themoviedb.org/3/movie/{tmdb_id}/credits?api_key={api_key}"
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        cast = data.get('cast', [])
        
        # Formatage des données : "Acteur (Rôle)"
        actor_roles = [
            f"{actor['name']} ({actor['character']})"
            for actor in cast[:10]  # Limitez par exemple aux 10 premiers acteurs
        ]
        return ", ".join(actor_roles)
    else:
        return None

#* Fonction pour récupérer les mots-clés d'un film par ID
def get_movie_keywords(movie_id):
    url_keywords = f'https://api.themoviedb.org/3/movie/{movie_id}/keywords?api_key={api_key}&language=en-US'
    response_keywords = requests.get(url_keywords)
    keywords_data = response_keywords.json()
    keywords = [kw['name'] for kw in keywords_data.get('keywords', [])]
    return keywords

def categorize_years(year):
    if 1970 <= year <= 1979:
        return "70°s"
    if 1980 <= year <= 1989:
        return "80°s"
    if 1990 <= year <= 1999:
        return "90°s"
    if 2000 <= year <= 2009:
        return "2000"
    if 2010 <= year <= 2019:
        return "2010"
    if 2020 <= year <= 2029:
        return "2020"

def categorize_times(duree):
    if duree < 100:
        return "Court"
    elif 100 <= duree <= 200:
        return "Moyen"
    else:
        return "Long"
    
def categorize_votes(votes):
    if votes < 50000:
        return "Connu"
    elif 50000 <= votes < 200000:
        return "Populaire"
    elif 200000 <= votes < 1000000:
        return "Très populaire"
    else:
        return "Blockbuster"

## **Filtrage Général**

In [7]:

# S'assurer que la colonne 'Langues Parlées' est correctement formatée
df_final['Langues Parlées'] = df_final['Langues Parlées'].astype(str)

# Critères de sélection avec contains
df_final = df_final[
    # Films en français avec plus de 2000 votes et popularité > 1.5
    ((df_final['Votes imdb'] > 8000) & (df_final['Popularité'] > 5) & (df_final['Langue Originale'].str.contains('fr', case=False, na=False))) |
    # Films avec des langues parlées contenant 'French', plus de 10000 votes et popularité > 5
    ((df_final['Votes imdb'] > 15000) & (df_final['Popularité'] > 7.5) & (df_final['Langues Parlées'].str.contains('French', case=False, na=False))) |
    # Films en anglais avec plus de 20000 votes et popularité > 10
    ((df_final['Votes imdb'] > 20000) & (df_final['Popularité'] > 10) & (df_final['Langue Originale'].str.contains('en', case=False, na=False))) |
    # Films populaires dans toutes les langues avec au moins 30000 votes et popularité > 20
    ((df_final['Votes imdb'] > 50000) & (df_final['Popularité'] > 25))
]

#* On conserve uniquement les films ayant des notes IMDb et TMDb supérieures à 4.
df_final = df_final[
    ~((df_final['Note imdb'] < 4) & (df_final['Note tmdb'] < 4) & (df_final['Date de Sortie'] < '2025-01-01'))
]

#* Exclusion des films ayant une note IMDb inférieure à 8 et sortis avant l'année 2000.
df_final = df_final[
    ~((df_final['Note imdb'] < 8) & (df_final['Date de Sortie'] < '2000-01-01'))
]

#* On élimine les doublons en se basant sur le titre original.
df_final = df_final.drop_duplicates(subset='Titre Original')

#* Création d'une copie explicite pour éviter des avertissements potentiels de pandas.
df_final = df_final.copy()

#* Conversion de la colonne 'Durée' en format numérique.
df_final['Durée'] = pd.to_numeric(df_final['Durée'], errors='coerce')

In [13]:

df_final['Réputation'] = df_final['Votes imdb'].apply(categorize_votes)
df_final['Métrage'] = df_final['Durée'].apply(categorize_times)
df_final['Mots-Clés'] = df_final['ID tmdb'].apply(get_movie_keywords)
df_final['Acteurs'] = df_final['ID tmdb'].astype(str).apply(get_cast_with_roles)
df_final = remplacer_synopsis_par_overview_fr(df_final)

df_final['Décennie'] = df_final['Date de Sortie'].apply(lambda x: categorize_years(int(str(x)[:4])) if isinstance(x, str) else x)
df_final['Genre Principal'] = df_final['Genres'].apply(lambda x: x.split(',')[0] if isinstance(x, str) else x)

#* On exclut les films dont le genre principal est 'Documentary'.
df_final = df_final[df_final['Genre Principal'] != 'Documentary']

#* On repositionne les colonnes pour une meilleure lisibilité et cohérence.
df_final = df_final[
    ['ID imdb', 'ID tmdb', 'Titre Original', 'Titre Français', 'Réalisateur(s)', 'Acteurs',
     'Budget', 'Genres', 'Mots-Clés', 'Genre Principal', 'Date de Sortie', 'Décennie',
     'Langue Originale', 'Langues Parlées', 'Synopsis', 'Popularité', 'Réputation',
     'Affiche', 'Image de Fond', 'Durée', 'Métrage',
     'Note tmdb', 'Votes tmdb', 'Note imdb', 'Votes imdb',
     'Compagnies de Production', 'Pays de Production', 'Box Office']
]


In [15]:
df_final['Mots-Clés'] = df_final['Mots-Clés'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)
df_final['Mots-Clés'] = df_final['Mots-Clés'].str.replace(r"['\[\]]", '', regex=True)

## **Exportation**

In [17]:

#! === EXPORT ===

df_final.to_csv("../data/processed/df_movie_cleaned.csv", index=False)

## **Rendu :**

In [3]:
import pandas as pd

df = pd.read_csv("../data/processed/df_movie_cleaned.csv")